Dataset from: https://zindi.africa/competitions/digital-africa-plantation-counting-challenge

In [ ]:
!pip install efficientnet_pytorch   

import os

import albumentations as A
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from PIL import Image

import torch
import torch.nn.functional as F

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=c4c8e3e71ef05f7b90dd5c4dbf1aad4ae668aa1866d61a513c887f62cf80cac7
  Stored in directory: /root/.cache/pip/wheels/29/16/24/752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
Successfully built efficientnet_pytorch


In [ ]:
!nvidia-smi

Thu Mar  9 11:38:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    24W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Train = pd.read_csv('/content/drive/My Drive/ZindiPlantationCounting/Train.csv')
Test = pd.read_csv('/content/drive/My Drive/ZindiPlantationCounting/Test.csv')
SampleSubmission = pd.read_csv('/content/drive/My Drive/ZindiPlantationCounting/SampleSubmission.csv')

In [ ]:
SampleSubmission

,ImageId,Target
0,Id_ohk78h9ld8.png,0
1,Id_eeyj2u4j7y.png,0
2,Id_wsd7vx2ifa.png,0
3,Id_6vfneamaoh.png,0
4,Id_9wil3575fv.png,0
...,...,...
853,Id_lmvuv1pm3a.png,0
854,Id_ez9lb2o6b1.png,0
855,Id_jeou44iven.png,0
856,Id_341bsipcnk.png,0


In [ ]:
train_split = 0.85
n_train = int(len(Train) * 0.85)
n_test = int(len(Train) - n_train)
print(f"training images: {n_train}, testing images: {n_test}")

training images: 1701, testing images: 301


In [ ]:
class d(Dataset):
    def __init__(self, csv_file, root_images, is_train=True, is_inference=False, transform=None ):
        
        self.root_images = root_images
        self.transform   = transform
        self.is_inference= is_inference
        
        if is_inference:
            self.csv_file = csv_file
        else:
            if is_train:
                self.csv_file = csv_file[:int(csv_file.shape[0]*0.85)].reset_index(drop=True)
            else:
                self.csv_file = csv_file[int(csv_file.shape[0]*0.85):].reset_index(drop=True)        
        
    def __len__(self):
        return self.csv_file.shape[0]
    
    def __getitem__(self, index):
        
        root_and_dir = self.csv_file['ImageId'][index]
        if not self.is_inference:
            label = self.csv_file['Target'][index]
        
        image = np.array(Image.open(os.path.join(self.root_images, root_and_dir)).convert('RGB'))
        
        if self.transform is not None:
            augmentations = self.transform(image=image)
            image         = augmentations['image']
            
        if not self.is_inference:
            return image, torch.as_tensor(label)
    
        return image

In [ ]:
LR = 1e-3 # learning rate
BS = 2 # batch size
NE = 100 # number of epochs
H  = 1024 # image height
W  = 1024 # image width
train_file = Train
test_file  = Test
image_path = '/content/drive/My Drive/ZindiPlantationCounting/TreeImages'

In [ ]:
normalize = A.Normalize(
    mean = [0.5,0.5,0.5],
    std  = [0.5,0.5,0.5], max_pixel_value=255
)

train_transform = A.Compose([
    A.Resize(H,W),
    A.Blur(p=0.3),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    normalize,
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(H,W),
    normalize,
    ToTensorV2(),
])

In [ ]:
train_ds     = d(train_file, image_path, is_train=True, is_inference=False, transform=train_transform )
train_loader = DataLoader(train_ds, batch_size=BS, shuffle=True)

val_ds     = d(train_file, image_path, is_train=False, is_inference=False, transform=val_transform )
val_loader = DataLoader(val_ds, batch_size=BS, shuffle=False)

test_ds     = d(test_file, image_path, is_train=False, is_inference=True, transform=val_transform )
test_loader = DataLoader(test_ds, batch_size=BS, shuffle=False)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b3')
        self.fc    = nn.Linear(1000, 1)
        self.relu  = nn.ReLU()
        
    def forward(self, image):
        x = self.model(image)
        x = self.fc(x)
        return self.relu(x)

In [ ]:
def check_acc(loader, model):
    loss = 0
    model.eval()
    with torch.no_grad():
        for x,y in tqdm(loader):
            x = x.to('cuda').to(torch.float32)
            y = y.to(torch.float).unsqueeze(1)
            
            preds = model(x)
            loss += np.sqrt(mean_squared_error(preds.cpu(), y))
    print(f'Loss function: {loss/len(loader)}')
    model.train()
    return loss/len(loader)
    
def save_checkpoint(state, filename):
    print('--> Saving checkpoint')
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print('--> Loading checkpoint')
    model.load_state_dict(checkpoint['state_dict'])

In [ ]:
def train_fn(loader, model, opt, loss_fn):
    for x,y in tqdm(loader):
        x = x.to('cuda').to(torch.float32)
        y = y.to(torch.float).unsqueeze(1).to('cuda')

        preds = model(x).to(torch.float)
        
        loss = loss_fn(preds, y)
        
        model.zero_grad()
        loss.backward()
        opt.step()

In [ ]:
loss_fn = nn.MSELoss().to('cuda')
model   = Net().to('cuda')
opt     = optim.Adam(model.parameters(), lr=LR)
loss    = 999999999
es      = 0


#check_acc(val_loader, model)
for epoch in range(NE):
    print('------------------------------- Epoch: '+str(epoch))
    
    train_fn(train_loader, model, opt, loss_fn)
    new_loss = check_acc(val_loader, model)    
    if new_loss < loss:
        loss = new_loss
        es   = 0
        checkpoint = {
            'state_dict': model.state_dict(),
            'optimizer':  opt.state_dict()
        }
        save_checkpoint(checkpoint, filename='baseline.pth.tar')
    
    else:
        es +=1
    
    if es == 3 :
        break

Loaded pretrained weights for efficientnet-b3
------------------------------- Epoch: 0


100%|██████████| 151/151 [01:58<00:00,  1.27it/s]


Loss function: 10.663227855567111
--> Saving checkpoint
------------------------------- Epoch: 1


100%|██████████| 151/151 [00:39<00:00,  3.84it/s]


Loss function: 2.940116244386758
--> Saving checkpoint
------------------------------- Epoch: 2


...

Loss function: 5.042157772280868
------------------------------- Epoch: 3


100%|██████████| 151/151 [00:38<00:00,  3.90it/s]


Loss function: 3.4461799666088146
------------------------------- Epoch: 4


100%|██████████| 151/151 [00:38<00:00,  3.88it/s]

Loss function: 5.385007365649899


In [ ]:
load_checkpoint(torch.load('baseline.pth.tar'), model)

--> Loading checkpoint


In [ ]:
def inference(loader, model):
    model.eval()
    all_preds = np.array([])
    with torch.no_grad():
        for x in tqdm(loader):
            x = x.to('cuda').to(torch.float32)
            all_preds = np.append(all_preds, model(x).cpu())
            
    print('Done!')
    return all_preds                            

In [ ]:
preds = inference(test_loader, model)

100%|██████████| 429/429 [14:16<00:00,  2.00s/it]

Done!


In [ ]:
Test['Target'] = preds

NameError: ignored

In [ ]:
Test.to_csv('sub.csv', index=False)

In [ ]:
Test

,ImageId,Target
0,Id_ohk78h9ld8.png,0.500275
1,Id_eeyj2u4j7y.png,-0.536374
2,Id_wsd7vx2ifa.png,11.203323
3,Id_6vfneamaoh.png,5.154625
4,Id_9wil3575fv.png,15.350361
...,...,...
853,Id_lmvuv1pm3a.png,0.492969
854,Id_ez9lb2o6b1.png,34.963478
855,Id_jeou44iven.png,0.008229
856,Id_341bsipcnk.png,8.018290
